In [73]:
from collections import Counter
import ast
import os
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [74]:
data_path = "./data/listings/"
data = pd.DataFrame()
for file_name in os.listdir(data_path):
    data = pd.concat([data, pd.read_csv(f"{data_path}{file_name}", sep=",")], ignore_index=True)

In [75]:
data.sort_values(by="last_scraped", ascending=False, inplace=True)
data.drop_duplicates(subset=["id"], keep="first", inplace=True)

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74859 entries, 42845 to 37469
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            74859 non-null  int64  
 1   listing_url                                   74859 non-null  object 
 2   scrape_id                                     74859 non-null  int64  
 3   last_scraped                                  74859 non-null  object 
 4   source                                        74859 non-null  object 
 5   name                                          74850 non-null  object 
 6   description                                   73614 non-null  object 
 7   neighborhood_overview                         45659 non-null  object 
 8   picture_url                                   74859 non-null  object 
 9   host_id                                       74859 non-n

In [77]:
data["amenities"] = [ast.literal_eval(amenities) for amenities in data["amenities"]]

In [78]:
def normalize_amenity(amenity: str) -> str:
    amenity = amenity.lower()
    amenity = amenity.replace("-", " ")
    amenity = re.sub(r"[^\w\s]", " ", amenity)
    amenity = re.sub(r"[\s]{2,}", " ", amenity)
    amenity = amenity.strip()
    return amenity

In [79]:
amenities = [normalize_amenity(amenity) for amenities in data["amenities"] for amenity in amenities]
amenities = Counter(amenities)

In [80]:
# remove unpopular (not standardized) amenity values
MIN_AMENITY_COUNT = 200

origin_amenity_keys = len(amenities.keys())
origin_amenity_values_sum = sum(amenities.values())

amenities = {amenity: amenity_count for (amenity, amenity_count) in amenities.items() if amenity_count >= MIN_AMENITY_COUNT}

print(f"Removed {round(abs(len(amenities.keys()) / origin_amenity_keys - 1) * 100, 1)}% amenity keys "
      f"but only {round(abs(sum(amenities.values()) / origin_amenity_values_sum - 1) * 100, 1)}% of the set amenity values in total.")

Removed 97.4% amenity keys but only 1.5% of the set amenity values in total.


In [81]:
sorted(amenities.items(), key=lambda x:x[1], reverse=True)

[('kitchen', 68536),
 ('smoke alarm', 67430),
 ('long term stays allowed', 66561),
 ('essentials', 63462),
 ('wifi', 62247),
 ('hangers', 57286),
 ('iron', 56349),
 ('hair dryer', 52858),
 ('washer', 51539),
 ('hot water', 51223),
 ('heating', 49684),
 ('tv', 49351),
 ('dishes and silverware', 49149),
 ('free parking on premises', 47539),
 ('refrigerator', 46315),
 ('microwave', 45437),
 ('shampoo', 45111),
 ('cooking basics', 44851),
 ('bed linens', 41322),
 ('air conditioning', 41224),
 ('oven', 34956),
 ('stove', 34296),
 ('private entrance', 32307),
 ('dishwasher', 31827),
 ('fire extinguisher', 30693),
 ('dryer', 30576),
 ('free street parking', 28805),
 ('extra pillows and blankets', 28719),
 ('coffee maker', 27223),
 ('first aid kit', 27120),
 ('bbq grill', 25988),
 ('lockbox', 24277),
 ('hot water kettle', 22917),
 ('toaster', 22242),
 ('wine glasses', 20637),
 ('dining table', 20090),
 ('cleaning products', 19419),
 ('shower gel', 19135),
 ('dedicated workspace', 19055),
 ('fr

In [84]:
new_amenity_data = pd.concat([pd.Series(name=f"amenity_{amenity}",
                              data=[amenity in accomodation_amenities for accomodation_amenities in data["amenities"]]) 
                              for amenity in amenities.keys()], axis=1)
data = pd.concat([data, new_amenity_data], axis=1)

In [85]:
data.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,amenity_dishwasher,amenity_bbq grill,amenity_hangers,amenity_private entrance,amenity_coffee maker,amenity_microwave,amenity_drying rack for clothing,amenity_room darkening shades,amenity_bed linens,amenity_air conditioning,amenity_wine glasses,amenity_indoor fireplace,amenity_cooking basics,amenity_long term stays allowed,amenity_essentials,amenity_oven,amenity_hot water,amenity_bathtub,amenity_free street parking,amenity_hot water kettle,amenity_wifi,amenity_kitchen,amenity_high chair,amenity_hair dryer,amenity_toaster,amenity_gas stove,amenity_heating,amenity_single level home,amenity_lockbox,amenity_patio or balcony,amenity_washer,amenity_crib,amenity_board games,amenity_tv,amenity_iron,amenity_outdoor dining area,amenity_fireplace guards,amenity_laundromat nearby,amenity_shampoo,amenity_outdoor furniture,amenity_children s dinnerware,amenity_free parking on premises,amenity_smoke alarm,amenity_barbecue utensils,amenity_extra pillows and blankets,amenity_backyard,amenity_refrigerator,amenity_dishes and silverware,amenity_fire pit,amenity_nespresso machine,amenity_shower gel,amenity_clothing storage walk in closet and wardrobe,amenity_shared patio or balcony,amenity_breakfast,amenity_beachfront,amenity_shared pool,amenity_first aid kit,amenity_fire extinguisher,amenity_shared garden or backyard,amenity_host greets you,amenity_cleaning products,amenity_stove,amenity_bikes,amenity_pocket wifi,amenity_private patio or balcony,amenity_dining table,amenity_freezer,amenity_beach essentials,amenity_private hot tub,amenity_ceiling fan,amenity_baking sheet,amenity_game console,amenity_private fenced garden or backyard,amenity_lake access,amenity_children s books and toys,amenity_body soap,amenity_smart lock,amenity_mini fridge,amenity_dedicated workspace,amenity_outdoor shower,amenity_ping pong table,amenity_private garden or backyard,amenity_baby safety gates,amenity_security cameras on property,amenity_ac split type ductless system,amenity_dryer,amenity_sound system,amenity_portable fans,amenity_safe,amenity_elevator,amenity_electric stove,amenity_children s books and toys for ages 0 2 years old 2 5 years old 5 10 years old and 10 years old,amenity_clothing storage wardrobe,amenity_rice maker,amenity_clothing storage,amenity_tv with netflix,amenity_pack n play travel crib,amenity_ethernet connection,amenity_radiant heating,amenity_carbon monoxide alarm,amenity_kayak,amenity_free driveway parking on premises 1 space,amenity_ev charger,amenity_central air conditioning,amenity_conditioner,amenity_clothing storage walk in closet,amenity_central heating,amenity_free washer in building,amenity_pool,amenity_clothing storage closet,amenity